In [1]:
### replicate results of EVmutation with the PABP_YEAST dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from model import CouplingsModel
import tools
import scipy
from pathlib import Path
from collections import OrderedDict
# biopython SeqIO
from Bio import SeqIO
from sklearn.metrics import roc_auc_score
import scipy

In [2]:
### specify which msa sequences to train the model on
START_IDX = 0
END_IDX = 100000
FILE_SUFFIX = (str(START_IDX)+'_'+ str(END_IDX)) if START_IDX != 0 else str(END_IDX)

In [3]:
# fasta_sequences = SeqIO.parse(open("BLAT_ECOLX/data/BLAT_ECOLX.a2m"),'fasta')
# with open(f"BLAT_ECOLX/data/BLAT_ECOLX_{FILE_SUFFIX}.a2m", "w+") as out_file:
#     cnt = 0
#     seq = []
#     for fasta in fasta_sequences:
#         if cnt >= END_IDX:
#             break
#         if START_IDX <= cnt:
#             seq.append(fasta)
#         cnt += 1

#     SeqIO.write(seq, out_file, "fasta")

In [4]:
regularization = 0.01 * 20 * (214)
print(regularization)

42.800000000000004


In [5]:
### training code for model
#!plmc/bin/plmc -o BLAT_ECOLX/model/BLAT_ECOLX.model_params -c BLAT_ECOLX/model/BLAT_ECOLX.txt -f BLAT_ECOLX -le 42.8 -lh 0.01 -m 200 -t 0.2 -g BLAT_ECOLX/data/BLAT_ECOLX.a2m

In [6]:
c = CouplingsModel(f"BLAT_ECOLX/model/BLAT_ECOLX.model_params")

In [16]:
data = pd.read_csv(
    "BLAT_ECOLX/data/BLAT_ECOLX_RES.csv", sep=",", comment="#"
)

In [17]:
data_pred = tools.predict_mutation_table(
        c, data, "effect_prediction_epistatic"
    )

H24S
check
H24C
check
H24D
check
H24E
check
H24F
check
H24G
check
H24I
check
H24A
check
H24K
check
H24N
check
H24P
check
H24Q
check
H24R
check
H24Y
check
H24W
check
H24L
check
H24V
check
H24T
check
P25W
check
P25A
check
P25C
check
P25D
check
P25E
check
P25F
check
P25Y
check
P25H
check
P25I
check
P25G
check
P25L
check
P25M
check
P25N
check
P25Q
check
P25R
check
P25S
check
P25T
check
P25K
check
P25V
check
E26L
check
E26A
check
E26D
check
E26G
check
E26I
check
E26K
check
E26Q
check
E26T
check
E26V
check
E26R
check
T27Y
check
T27W
check
T27V
check
T27S
check
T27Q
check
T27P
check
T27N
check
T27M
check
T27L
check
T27I
check
T27A
check
T27C
check
T27D
check
T27E
check
T27F
check
T27G
check
T27H
check
T27K
check
T27R
check
L28A
check
L28C
check
L28E
check
L28G
check
L28H
check
L28I
check
L28K
check
L28N
check
L28P
check
L28M
check
L28R
check
L28S
check
L28T
check
L28V
check
L28Q
check
V29D
check
V29M
check
V29E
check
V29G
check
V29I
check
V29L
check
V29N
check
V29T
check
V29Q
check
V29R
check

In [18]:
tools.extract_mutations('W286D')

[(286, 'W', 'D')]

In [19]:
c.delta_hamiltonian(tools.extract_mutations('F70W'))

array([-5.60967346, -0.77988716, -4.8297863 ])

In [22]:
data[data['mutant'] == 'W286D']['EVmutation']

4782   -7.397216
Name: EVmutation, dtype: float64

In [ ]:
mutant,mutated_sequence,DMS_score,DMS_score_bin,Tranception_L_no_retrieval,Tranception_S_retrieval,Tranception_M_retrieval,Tranception_L_retrieval,EVE_single,EVE_ensemble,MSA_Transformer_single,MSA_Transformer_ensemble,ESM1v_single,ESM1v_ensemble,Wavenet,DeepSequence_single,DeepSequence_ensemble,Site_Independent,EVmutation,RITA_s,RITA_m,RITA_l,RITA_xl,RITA_ensemble,Progen2_small,Progen2_medium,Progen2_base,Progen2_large,Progen2_xlarge,Progen2_ensemble,Ensemble_Tranception_EVE